In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import numpy as np
import os

from prepare.classes import LunaPatchDataset, Advanced3DAugment
from model.model import Luna3DCNN  # Update path as needed

# --- Hyperparams ---
BATCH_SIZE = 16
NUM_EPOCHS = 20
LR = 1e-4
#PATCH_CSV = "./output/training_balanced.csv"
CHECKPOINT_PATH = "best_model.pt"
OUTPUT_PATH = r"D:\output"
PATCH_CSV = OUTPUT_PATH + r"\training_balanced.csv"


# --- Dataset ---
full_dataset = LunaPatchDataset(csv_file=PATCH_CSV, transform=None)

# Split 80/20
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply augmentation only to training set
train_dataset.dataset.transform = Advanced3DAugment()
val_dataset.dataset.transform = None

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# --- Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Luna3DCNN().to(device)

# --- Loss & Optimizer ---
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=2)

# --- Training Functions ---

def run_epoch(model, loader, criterion, optimizer=None):
    is_train = optimizer is not None
    model.train() if is_train else model.eval()

    losses, all_labels, all_preds = [], [], []

    for x, y in tqdm(loader, desc="Train" if is_train else "Val"):
        x = x.to(device).float()
        y = y.to(device).float().view(-1, 1)

        if is_train:
            optimizer.zero_grad()

        logits = model(x)
        loss = criterion(logits, y)

        if is_train:
            loss.backward()
            optimizer.step()

        preds = torch.sigmoid(logits).detach().cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(y.cpu().numpy())
        losses.append(loss.item())

    y_true = np.array(all_labels)
    y_pred = np.array(all_preds)

    auc = roc_auc_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred > 0.5)
    precision = precision_score(y_true, y_pred > 0.5)
    recall = recall_score(y_true, y_pred > 0.5)
    f1 = f1_score(y_true, y_pred > 0.5)

    return np.mean(losses), acc, auc, precision, recall, f1

# --- Train Loop ---

best_auc = 0

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")

    train_loss, train_acc, train_auc, _, _, _ = run_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc, val_auc, val_precision, val_recall, val_f1 = run_epoch(model, val_loader, criterion)

    print(f"[Train] Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | AUC: {train_auc:.4f}")
    print(f"[Val]   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | AUC: {val_auc:.4f} | F1: {val_f1:.4f} | P: {val_precision:.4f} | R: {val_recall:.4f}")

    scheduler.step(val_auc)

    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), CHECKPOINT_PATH)
        print(f"✅ Saved new best model with AUC {best_auc:.4f}")



Epoch 1/20


Val: 100%|█████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 69.87it/s]


[Train] Loss: 0.7032 | Acc: 0.4970 | AUC: 0.5341
[Val]   Loss: 0.6929 | Acc: 0.5122 | AUC: 0.8537 | F1: 0.6774 | P: 0.5122 | R: 1.0000
✅ Saved new best model with AUC 0.8537

Epoch 2/20


Val: 100%|█████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 72.07it/s]


[Train] Loss: 0.6933 | Acc: 0.4970 | AUC: 0.4766
[Val]   Loss: 0.6930 | Acc: 0.5122 | AUC: 0.8510 | F1: 0.6774 | P: 0.5122 | R: 1.0000

Epoch 3/20


...

[Train] Loss: 0.6932 | Acc: 0.4970 | AUC: 0.4847
[Val]   Loss: 0.6931 | Acc: 0.5122 | AUC: 0.8323 | F1: 0.6774 | P: 0.5122 | R: 1.0000

Epoch 19/20


Val: 100%|█████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 74.42it/s]


[Train] Loss: 0.6932 | Acc: 0.4970 | AUC: 0.6030
[Val]   Loss: 0.6931 | Acc: 0.5122 | AUC: 0.8394 | F1: 0.6774 | P: 0.5122 | R: 1.0000

Epoch 20/20


Val: 100%|█████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 73.73it/s]

[Train] Loss: 0.6932 | Acc: 0.4970 | AUC: 0.5620
[Val]   Loss: 0.6931 | Acc: 0.5122 | AUC: 0.8570 | F1: 0.6774 | P: 0.5122 | R: 1.0000
